In [ ]:
# ✅ 1. Install required libraries
!pip install -q transformers datasets peft accelerate

In [ ]:
# ✅ 2. Import everything
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import torch


In [ ]:
# ✅ 3. Load a small base model (Tiny GPT-2)
model_id = "sshleifer/tiny-gpt2"


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Add a padding token to the tokenizer if it doesn't have one.
# Tiny GPT-2 tokenizer doesn't have a default pad token, so we use the eos_token.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# ✅ 4. Apply LoRA config
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["c_attn"],  # GPT-2's attention layer
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

In [ ]:
model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
# ✅ 5. Load a tiny dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")

In [ ]:
# ✅ 6. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)



In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/367 [00:00<?, ? examples/s]

In [ ]:
# ✅ 7. Prepare training arguments
training_args = TrainingArguments(
    output_dir="./tiny-gpt2-lora",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=50,
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
)

In [ ]:
# ✅ 8. Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# ✅ 9. Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-19-33101e81b739>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# ✅ 10. Start training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karishmac91 (karishmac91-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,10.545900
20,10.417200
30,10.471000
40,10.394500
50,10.496100
60,10.206500
70,10.562900
80,10.463400
90,10.524400


/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:4064: UserWarning: mtime may not be reliable on this filesystem, falling back to numerical ordering
  warnings.warn("mtime may not be reliable on this filesystem, falling back to numerical ordering")


TrainOutput(global_step=92, training_loss=10.458584868389627, metrics={'train_runtime': 268.1669, 'train_samples_per_second': 1.369, 'train_steps_per_second': 0.343, 'total_flos': 60880896.0, 'train_loss': 10.458584868389627, 'epoch': 1.0})

In [ ]:
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.json',
 'fine_tuned_model/merges.txt',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")


In [ ]:
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained("sshleifer/tiny-gpt2")



In [ ]:
input_text = "Machine learning and AI is transforming industries by"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate with some max tokens
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,        # enable sampling to add randomness
    top_p=0.9,             # nucleus sampling parameter
    temperature=0.8        # control creativity: lower is conservative, higher is creative
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Machine learning and AI is transforming industries bydit intermittenthibit Observ dispatchreement Motorola intermittentohoohoJD Daniel Rh autonomy vendors ONEreement Hancock credibility reviewingRocket circumcised Money ONE credibility scalpJD circumcised antibiotic Prob stairs Probpress Habit Daniel conservation ONE scalpScene vendorsmediately Jr TAtingSceneohohibitiken hauled Jr
